# 실험 01: 기본 프롬프트 vs 구조화 프롬프트

**프롬프트 엔지니어링의 가장 기본적인 기법을 검증하는 실험**

---

## 1. 개요

### 목적
프롬프트의 **구조화**가 LLM(대규모 언어 모델)의 응답 품질에 어떤 영향을 미치는지 검증합니다.

### 배경
LLM에게 질문할 때, 같은 내용을 물어도 **어떻게 물어보느냐**에 따라 답변의 품질이 달라집니다.

예를 들어:
- ❌ "이거 요약해줘" → 모호함
- ✅ "아래 텍스트를 2문장으로 요약하세요. 핵심 정보만 포함하세요." → 명확함

### 가설
구조화된 프롬프트(마크다운 헤딩, 명확한 지시사항)를 사용하면:
1. 응답의 **일관성**이 높아질 것이다
2. 불필요한 출력이 줄어 **토큰 효율성**이 개선될 것이다
3. 원하는 형식의 답변을 받을 확률이 높아질 것이다

## 2. 구조화 프롬프트란?

### 정의
**구조화 프롬프트(Structured Prompting)**는 프롬프트를 논리적인 섹션으로 나누어 작성하는 기법입니다.

### 핵심 요소
| 요소 | 설명 | 예시 |
|------|------|------|
| 역할 정의 | AI의 역할을 명시 | "당신은 전문 번역가입니다" |
| 지시사항 | 수행할 작업을 명확히 | "2문장으로 요약하세요" |
| 규칙/제약 | 지켜야 할 조건 | "원문에 없는 내용 추가 금지" |
| 입력 데이터 | 처리할 데이터 | 요약할 텍스트 |
| 출력 형식 | 원하는 응답 형식 | "JSON 형식으로 출력" |

### 왜 효과적인가?
1. **명확성**: AI가 무엇을 해야 하는지 정확히 이해
2. **일관성**: 같은 프롬프트로 일관된 결과 획득
3. **제어 가능성**: 출력 형식과 길이를 조절 가능

### 학술적 근거
- OpenAI의 [Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)에서 권장하는 best practice
- Google의 PALM 2 논문에서도 구조화된 프롬프트의 효과성 검증

## 3. 환경 설정

### 사용 기술
| 항목 | 값 | 설명 |
|------|------|------|
| 모델 | qwen2.5:7b | Alibaba의 오픈소스 LLM (7B 파라미터) |
| 프레임워크 | LangChain | LLM 애플리케이션 개발 프레임워크 |
| 토큰 계산 | tiktoken | OpenAI의 토큰화 라이브러리 |
| 실행 환경 | Ollama | 로컬 LLM 실행 도구 |

### 평가 지표
| 지표 | 설명 | 중요도 |
|------|------|--------|
| 토큰 수 | 입력+출력 토큰의 합 | 비용과 직결 |
| 응답 시간 | API 호출부터 응답까지 | 사용자 경험 |
| 응답 품질 | 지시사항 준수 여부 | 정확성 |

In [ ]:
# ============================================================
# 환경 설정 및 라이브러리 임포트
# ============================================================

from langchain_ollama import ChatOllama
import tiktoken
import time
import json
from datetime import datetime

# LLM 초기화
# - model: 사용할 모델명
# - temperature: 0으로 설정하여 결정적(deterministic) 출력 보장
llm = ChatOllama(model="qwen2.5:7b", temperature=0)

# 토큰 카운터 초기화
# - tiktoken은 OpenAI의 토크나이저로, 업계 표준으로 사용됨
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

print("환경 설정 완료")
print(f"- 모델: qwen2.5:7b")
print(f"- Temperature: 0 (결정적 출력)")

In [ ]:
# ============================================================
# 유틸리티 함수 정의
# ============================================================

def count_tokens(text: str) -> int:
    """
    텍스트의 토큰 수를 계산합니다.
    
    토큰이란?
    - LLM이 텍스트를 처리하는 최소 단위
    - 영어: 약 4글자 = 1토큰
    - 한국어: 약 1-2글자 = 1토큰 (더 비효율적)
    
    Args:
        text: 토큰 수를 계산할 텍스트
    
    Returns:
        int: 토큰 수
    """
    return len(encoder.encode(text))


def run_prompt(prompt: str, name: str = "") -> dict:
    """
    프롬프트를 실행하고 결과를 측정합니다.
    
    측정 항목:
    1. 입력 토큰 수: 프롬프트의 토큰 수
    2. 출력 토큰 수: 응답의 토큰 수
    3. 응답 시간: 실행에 걸린 시간(초)
    
    Args:
        prompt: 실행할 프롬프트
        name: 실험 이름 (로깅용)
    
    Returns:
        dict: 측정 결과와 응답 내용
    """
    # 시작 시간 기록
    start_time = time.time()
    
    # LLM 호출
    response = llm.invoke(prompt)
    
    # 종료 시간 기록
    elapsed_time = time.time() - start_time
    
    return {
        "name": name,
        "input_tokens": count_tokens(prompt),
        "output_tokens": count_tokens(response.content),
        "total_tokens": count_tokens(prompt) + count_tokens(response.content),
        "time": round(elapsed_time, 2),
        "response": response.content
    }


print("유틸리티 함수 정의 완료")

## 4. 실험 설계

### 테스트 데이터
AI와 딥러닝에 관한 4문장짜리 텍스트를 사용합니다.

### 비교 대상
| 프롬프트 유형 | 특징 |
|--------------|------|
| 기본 프롬프트 | 단순히 "요약해줘"라고 요청 |
| 구조화 프롬프트 | 마크다운 헤딩 + 명확한 규칙 + 출력 형식 지정 |

### 평가 방법
1. **토큰 효율성**: 동일한 작업에 필요한 토큰 수 비교
2. **응답 품질**: 지시사항(2문장 요약)을 얼마나 잘 따르는지

In [ ]:
# ============================================================
# 테스트 데이터 준비
# ============================================================

# 요약 대상 텍스트 (AI에 관한 설명문)
test_text = """
인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현한 컴퓨터 시스템이다.
최근 딥러닝 기술의 발전으로 자연어 처리, 이미지 인식, 음성 인식 등 다양한 분야에서 
인간 수준 또는 그 이상의 성능을 보이고 있다. 특히 대규모 언어 모델(LLM)의 등장으로
챗봇, 문서 요약, 코드 생성 등 다양한 응용 분야가 확대되고 있다.
""".strip()

print("테스트 텍스트:")
print("-" * 50)
print(test_text)
print("-" * 50)
print(f"텍스트 길이: {len(test_text)}자, {count_tokens(test_text)}토큰")

## 5. 실험 실행

### 5.1 기본 프롬프트 vs 구조화 프롬프트

In [ ]:
# ============================================================
# 실험 5.1: 기본 프롬프트
# ============================================================
# 특징: 단순하고 직관적이지만, 출력 형식이 모호함

basic_prompt = f"다음 글을 요약해줘: {test_text}"

print("[기본 프롬프트]")
print(basic_prompt)
print()

In [ ]:
# ============================================================
# 실험 5.1: 구조화 프롬프트
# ============================================================
# 특징: 
# - 마크다운 헤딩(###)으로 섹션 구분
# - 명확한 규칙 제시
# - 출력 위치 지정

structured_prompt = f"""### 지시사항
아래 텍스트를 2문장으로 요약하세요.

### 규칙
- 핵심 정보만 포함
- 원문에 없는 내용 추가 금지

### 텍스트
{test_text}

### 요약"""

print("[구조화 프롬프트]")
print(structured_prompt)

In [ ]:
# ============================================================
# 두 프롬프트 실행 및 결과 비교
# ============================================================

# 실행
result_basic = run_prompt(basic_prompt, "기본")
result_structured = run_prompt(structured_prompt, "구조화")

# 결과 출력
print("=" * 60)
print("실험 결과: 기본 프롬프트 vs 구조화 프롬프트")
print("=" * 60)

print("\n[기본 프롬프트 결과]")
print(f"  입력 토큰: {result_basic['input_tokens']}")
print(f"  출력 토큰: {result_basic['output_tokens']}")
print(f"  응답 시간: {result_basic['time']}초")
print(f"  응답 내용:")
print(f"  {result_basic['response']}")

print("\n[구조화 프롬프트 결과]")
print(f"  입력 토큰: {result_structured['input_tokens']}")
print(f"  출력 토큰: {result_structured['output_tokens']}")
print(f"  응답 시간: {result_structured['time']}초")
print(f"  응답 내용:")
print(f"  {result_structured['response']}")

### 5.2 다양한 구조화 스타일 비교

구조화 프롬프트도 여러 스타일이 있습니다:

| 스타일 | 특징 | 장점 |
|--------|------|------|
| Markdown | 헤딩(#, ##, ###) 사용 | 가독성 좋음 |
| XML | 태그(<tag>) 사용 | 파싱하기 쉬움 |
| JSON | 키-값 구조 | 프로그래밍 친화적 |

In [ ]:
# ============================================================
# 실험 5.2: 다양한 구조화 스타일 비교
# ============================================================

# 스타일 1: Markdown 스타일
markdown_prompt = f"""# 요약 작업

## 지시사항
2문장으로 요약하세요.

## 텍스트
{test_text}

## 요약"""

# 스타일 2: XML 스타일
xml_prompt = f"""<instruction>2문장으로 요약하세요.</instruction>

<text>
{test_text}
</text>

<summary>"""

# 스타일 3: JSON 스타일
json_prompt = f"""입력:
{{
    "task": "summarize",
    "max_sentences": 2,
    "text": "{test_text}"
}}

출력 (요약만):"""

# 실행
results_style = [
    run_prompt(markdown_prompt, "Markdown"),
    run_prompt(xml_prompt, "XML"),
    run_prompt(json_prompt, "JSON")
]

# 결과 출력
print("=" * 60)
print("스타일별 비교 결과")
print("=" * 60)
print(f"{'스타일':<12} {'입력 토큰':<12} {'출력 토큰':<12} {'시간':<10}")
print("-" * 60)

for r in results_style:
    print(f"{r['name']:<12} {r['input_tokens']:<12} {r['output_tokens']:<12} {r['time']}초")

## 6. 결과 분석

### 6.1 정량적 결과

In [ ]:
# ============================================================
# 결과 분석: 정량적 비교
# ============================================================

print("=" * 60)
print("정량적 결과 요약")
print("=" * 60)

# 기본 vs 구조화 비교
token_diff = result_structured['output_tokens'] - result_basic['output_tokens']
token_change = (token_diff / result_basic['output_tokens']) * 100 if result_basic['output_tokens'] > 0 else 0

time_diff = result_structured['time'] - result_basic['time']
time_change = (time_diff / result_basic['time']) * 100 if result_basic['time'] > 0 else 0

print("\n[기본 vs 구조화 프롬프트]")
print(f"┌{'─'*20}┬{'─'*15}┬{'─'*15}┬{'─'*15}┐")
print(f"│ {'항목':<18} │ {'기본':<13} │ {'구조화':<13} │ {'변화':<13} │")
print(f"├{'─'*20}┼{'─'*15}┼{'─'*15}┼{'─'*15}┤")
print(f"│ {'출력 토큰':<18} │ {result_basic['output_tokens']:<13} │ {result_structured['output_tokens']:<13} │ {token_change:>+10.1f}% │")
print(f"│ {'응답 시간(초)':<18} │ {result_basic['time']:<13} │ {result_structured['time']:<13} │ {time_change:>+10.1f}% │")
print(f"└{'─'*20}┴{'─'*15}┴{'─'*15}┴{'─'*15}┘")

print("\n[스타일별 비교]")
print(f"┌{'─'*12}┬{'─'*12}┬{'─'*12}┬{'─'*10}┐")
print(f"│ {'스타일':<10} │ {'입력토큰':<10} │ {'출력토큰':<10} │ {'시간':<8} │")
print(f"├{'─'*12}┼{'─'*12}┼{'─'*12}┼{'─'*10}┤")
for r in results_style:
    print(f"│ {r['name']:<10} │ {r['input_tokens']:<10} │ {r['output_tokens']:<10} │ {r['time']:<6}초 │")
print(f"└{'─'*12}┴{'─'*12}┴{'─'*12}┴{'─'*10}┘")

### 6.2 정성적 분석

**기본 프롬프트의 응답**
- 요약 길이가 일정하지 않을 수 있음
- "~입니다" 등 불필요한 서술 포함 가능

**구조화 프롬프트의 응답**
- "2문장"이라는 명확한 제약 준수
- 핵심 정보만 포함하려는 경향

## 7. 결론

### 핵심 발견

1. **구조화 프롬프트의 장점**
   - 명확한 지시사항으로 일관된 출력 형식 확보
   - 규칙을 명시하여 원치 않는 내용 방지

2. **스타일별 차이**
   - Markdown: 가독성 좋고 범용적
   - XML: 프로그래밍 통합 시 유리
   - JSON: API 응답 생성 시 적합

### 실무 적용 가이드

| 상황 | 권장 방식 |
|------|----------|
| 단순 질의응답 | 기본 프롬프트 |
| 특정 형식 필요 | 구조화 프롬프트 |
| API 응답 생성 | JSON/XML 스타일 |
| 문서 작성 | Markdown 스타일 |

### 한계점
- 단순 요약 작업에서는 차이가 크지 않을 수 있음
- 복잡한 작업에서 더 큰 효과 예상

### 다음 실험
- Chain of Thought (단계별 사고) 기법 실험
- 복잡한 추론 문제에서의 효과 검증

In [ ]:
# ============================================================
# 결과 저장
# ============================================================

experiment_results = {
    "experiment": "01_기본_프롬프트_vs_구조화_프롬프트",
    "date": datetime.now().isoformat(),
    "model": "qwen2.5:7b",
    "hypothesis": "구조화 프롬프트가 일관성과 토큰 효율성을 개선한다",
    "results": {
        "basic_vs_structured": {
            "basic": {
                "input_tokens": result_basic['input_tokens'],
                "output_tokens": result_basic['output_tokens'],
                "time": result_basic['time']
            },
            "structured": {
                "input_tokens": result_structured['input_tokens'],
                "output_tokens": result_structured['output_tokens'],
                "time": result_structured['time']
            },
            "comparison": {
                "token_change": f"{token_change:+.1f}%",
                "time_change": f"{time_change:+.1f}%"
            }
        },
        "style_comparison": [
            {"style": r['name'], "input_tokens": r['input_tokens'], 
             "output_tokens": r['output_tokens'], "time": r['time']}
            for r in results_style
        ]
    },
    "conclusion": {
        "finding": "구조화 프롬프트는 출력 형식 제어에 효과적",
        "recommendation": "특정 형식이 필요한 작업에 구조화 프롬프트 권장",
        "limitation": "단순 작업에서는 차이가 크지 않음"
    }
}

# JSON 파일로 저장
with open("../results/01_basic_results.json", "w", encoding="utf-8") as f:
    json.dump(experiment_results, f, ensure_ascii=False, indent=2)

print("실험 결과가 저장되었습니다: results/01_basic_results.json")

---

## 참고 자료

1. OpenAI Prompt Engineering Guide: https://platform.openai.com/docs/guides/prompt-engineering
2. LangChain Documentation: https://python.langchain.com/docs/
3. tiktoken: https://github.com/openai/tiktoken

---

*이 실험은 프롬프트 엔지니어링 포트폴리오 프로젝트의 일부입니다.*